In [1]:
import pandas as pd
import numpy as np
import re
import os
import import_ipynb
from modes_prepare_lib_лы import get_modes_dataframe
from datetime import datetime
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

importing Jupyter notebook from modes_prepare_lib_лы.ipynb


In [2]:
df_modes, df_valid = get_modes_dataframe()

129559
(129086, 38)
8989
(8958, 24)


In [5]:
# df_valid.columns, df_modes.columns

In [23]:
# df_valid['масса\nтн'] = df_valid['масса\nтн.1']

In [3]:
def change_to17(x):
    if x.year>2018:
        try:
            tmp=x
            tmp = tmp.replace(year=2017, month=x.month, day=x.day)
            return tmp
        except:
            tmp = tmp.replace(year=2017, month=x.month, day=28)
            return tmp
    else:
        return x

def union_modes():
    global df_modes
    global df_valid
    result=list(set(df_modes.columns) & set(df_valid.columns))
    df_modes = df_modes[result]
    df_modes['Дата термообработки'].describe()
    df_valid = df_valid[result]
    
    df = pd.concat([df_modes, df_valid])
    df.reset_index(drop=True, inplace=True)
    return df

df = union_modes()

In [4]:
timestamp_df = df[df['Дата термообработки'].apply(type)==pd._libs.tslib.Timestamp]
last_df = df[df['Дата термообработки'].apply(type)!=pd._libs.tslib.Timestamp]
last_df['Дата термообработки'] = last_df['Дата термообработки'].apply(
    lambda x:datetime.strptime(x, "%d.%m.%Y"))
timestamp_df['Дата термообработки'] = timestamp_df['Дата термообработки'].apply(
    pd._libs.tslib.Timestamp.to_pydatetime)
last_df['Дата термообработки'] = last_df['Дата термообработки'].apply(change_to17)
print(last_df[last_df['Дата термообработки'].apply(lambda x:x.year)>2018].shape)
last_df.shape
df = pd.concat([last_df, timestamp_df])
df.reset_index(drop=True, inplace=True)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

(0, 23)


In [26]:
# pd.DataFrame(df['Дата термообработки'].value_counts().sort_index().index).to_excel('1.xlsx')

In [5]:
def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

In [14]:
df = df[df['диаметр'].apply(isDigit)]
df = df[df['толщина стенки'].apply(isDigit)]
df.shape

(134982, 23)

In [10]:
df.to_excel('for_top_typesizes.xlsx')

In [29]:
# df['диаметр'].value_counts().sort_index()

In [10]:
df.columns

Index(['диаметр', '3 зона ВТР и уставка отпуск', '№ плавки', 'толщина стенки',
       'Скорость прохождения трубы через спрейер, м/с', '№ пакета',
       '4 зона ВТР и уставка отпуск', '№ партии', 'НД (гост, ту)',
       '2 зона по ВТР закалка', '3 зона по ВТР закалка',
       '2 зона ВТР и уставка отпуск', '1 зона по ВТР закалка',
       'шаг балок отпускная печь, сек', 'марка стали',
       'шаг балок закалочная печь, сек',
       'время выдержки в закалочной ванне, сек.', 't˚ C трубы после спреера',
       '5 зона ВТР и уставка отпуск', 't˚ C трубы после ванны',
       'Дата термообработки', '1 зона ВТР и уставка отпуск', 'масса\nтн'],
      dtype='object')

In [15]:
def conv2float(x):
    try:
        return float(x)
    except:
        return None

In [18]:
for x in df['шаг балок отпускная печь, сек']:
    try:
        x = float(x)
    except:
        print(x)

In [17]:
df['диаметр'] = df['диаметр'].astype(float)
df['толщина стенки'] = df['толщина стенки'].astype(float)
df['шаг балок закалочная печь, сек'] = df['шаг балок закалочная печь, сек'].apply(conv2float)
df['шаг балок отпускная печь, сек'] = df['шаг балок отпускная печь, сек'].apply(conv2float)

In [19]:
df.shape

(134982, 23)

In [20]:
def find_mark_typesize(mark, d, s):
    answ = df.copy()
    if mark!=None:
        answ = answ[answ['марка стали']==mark]
#     print(answ.shape)
    if d!=None:
        answ = answ[answ['диаметр']==d]
#     print(answ.shape)
    if s!=None:
        answ = answ[answ['толщина стенки']==s]
#     print(answ.shape)
    return answ

In [21]:
def find_mark_typesize_2018(mark, d, s):
    answ = df[df['Дата термообработки'].apply(lambda x: x.year == 2018)].copy()
    if mark!=None:
        answ = answ[answ['марка стали']==mark]
#     print(answ.shape)
    if d!=None:
        answ = answ[answ['диаметр']==d]
#     print(answ.shape)
    if s!=None:
        answ = answ[answ['толщина стенки']==s]
#     print(answ.shape)
    return answ

In [61]:
# список 15 самых массовых заказов типоразмеров по версии 2017г
top_typesizes = [
    ['30Г2',73.02,5.51],
    ['30Г2',177.8,9.2],
    ['30Г2',73,5.5],
    ['18ХМФБ',73.02,5.51],
    ['30Г2-8',73.02,5.51],
    ['30Г2',88.9,6.5],
    ['37Г2Ф',168.28,8.94],
    ['30Г2',88.9,6.45],
    ['37Г2Ф',177.8,9.2],
    ['30Г2',168.28,8.94],
    ['18ХМФБ',73,5.5],
    ['37Г2Ф',88.9,12.2],
    ['38Г2С',73,5.5],
    ['30Г2-8',73,5.5],

]


In [23]:
answ1 = find_mark_typesize('18ХМФБ',73.02,5.51)

In [31]:
answ1[answ1['Дата термообработки'] == answ1['Дата термообработки'].max()]

,диаметр,3 зона ВТР и уставка отпуск,№ плавки,толщина стенки,"Скорость прохождения трубы через спрейер, м/с",№ пакета,4 зона ВТР и уставка отпуск,№ партии,"НД (гост, ту)",2 зона по ВТР закалка,3 зона по ВТР закалка,2 зона ВТР и уставка отпуск,1 зона по ВТР закалка,"шаг балок отпускная печь, сек",марка стали,"шаг балок закалочная печь, сек","время выдержки в закалочной ванне, сек.",t˚ C трубы после спреера,5 зона ВТР и уставка отпуск,t˚ C трубы после ванны,Дата термообработки,1 зона ВТР и уставка отпуск,масса тн
129495,73.02,659,8f1869,5.51,0.7,403675797,659,804138,ТУ 21,870,880,670,870,27.0,18ХМФБ,27.0,,70.0,642,NaN,2018-04-11,670,4.79
129496,73.02,659,8f1869,5.51,0.7,403675730,659,804138,ТУ 21,870,880,670,870,27.0,18ХМФБ,27.0,,70.0,642,NaN,2018-04-11,670,4.77
129497,73.02,659,8f1869,5.51,0.7,403692843,659,804140,ТУ 21,870,880,670,870,27.0,18ХМФБ,27.0,,70.0,642,NaN,2018-04-11,670,3.63
129498,73.02,659,8f1869,5.51,0.7,403693075,659,804140,ТУ 21,870,880,670,870,27.0,18ХМФБ,27.0,,70.0,642,NaN,2018-04-11,670,3.06
129499,73.02,659,8f1869,5.51,0.7,403676479,659,804140,ТУ 21,870,880,670,870,27.0,18ХМФБ,27.0,,70.0,642,NaN,2018-04-11,670,4.38
129500,73.02,659,8f1869,5.51,0.7,403676220,659,804140,ТУ 21,870,880,670,870,27.0,18ХМФБ,27.0,,70.0,642,NaN,2018-04-11,670,3.33
129501,73.02,668,8f1872,5.51,0.7,403691981,668,804139,ТУ 21,870,880,680,870,27.0,18ХМФБ,27.0,,70.0,652,NaN,2018-04-11,680,4.78
129502,73.02,668,8f1872,5.51,0.7,403676388,668,804139,ТУ 21,870,880,680,870,27.0,18ХМФБ,27.0,,70.0,652,NaN,2018-04-11,680,4.76
129503,73.02,668,8f1872,5.51,0.7,403676300,668,804139,ТУ 21,870,880,680,870,27.0,18ХМФБ,27.0,,70.0,652,NaN,2018-04-11,680,3.04


In [141]:
top = pd.read_excel('top_typesizes_2017.xlsx')

In [142]:
top=top.sort_values('масса\nтн', ascending=False)

In [143]:
top_ls = top.head(30).copy()

In [144]:
top_ls.reset_index(inplace=True, drop=True)
top_ls

,марка стали,диаметр,толщина стенки,масса тн
0,30Г2,73.02,5.51,11970.149100
1,30Г2,177.80,9.20,10077.884038
2,30Г2,73.00,5.50,7731.340000
3,18ХМФБ,73.02,5.51,7616.750000
4,30Г2-8,73.02,5.51,4899.125000
5,30Г2,88.90,6.50,3185.250000
6,37Г2Ф,168.28,8.94,2975.130000
7,30Г2,88.90,6.45,2074.510000
8,37Г2Ф,177.80,9.20,2054.030000
9,30Г2,168.28,8.94,2029.170000


In [149]:
# поиск топовых типоразмеров в файлах за 2018г для валидации
for i in range(0,len(top_ls)):
    print(find_mark_typesize_2018(top_ls.iloc[i,:][0], top_ls.iloc[i,:][1], top_ls.iloc[i,:][2]).shape, '\n', top_ls.iloc[i, :])

(416, 23) 
 марка стали          30Г2
диаметр             73.02
толщина стенки       5.51
масса\nтн         11970.1
Name: 0, dtype: object
(349, 23) 
 марка стали          30Г2
диаметр             177.8
толщина стенки        9.2
масса\nтн         10077.9
Name: 1, dtype: object
(373, 23) 
 марка стали          30Г2
диаметр                73
толщина стенки        5.5
масса\nтн         7731.34
Name: 2, dtype: object
(21, 23) 
 марка стали        18ХМФБ
диаметр             73.02
толщина стенки       5.51
масса\nтн         7616.75
Name: 3, dtype: object
(122, 23) 
 марка стали        30Г2-8
диаметр             73.02
толщина стенки       5.51
масса\nтн         4899.12
Name: 4, dtype: object
(256, 23) 
 марка стали          30Г2
диаметр              88.9
толщина стенки        6.5
масса\nтн         3185.25
Name: 5, dtype: object
(0, 23) 
 марка стали         37Г2Ф
диаметр            168.28
толщина стенки       8.94
масса\nтн         2975.13
Name: 6, dtype: object
(65, 23) 
 марка стали        

In [160]:
# якобы минимальные шаги топовых типоразмеров
for i in range(0,len(top_ls)):
    answ = find_mark_typesize(top_ls.iloc[i,:][0], top_ls.iloc[i,:][1], top_ls.iloc[i,:][2])
    print(answ['шаг балок отпускная печь, сек'].value_counts(), answ['шаг балок закалочная печь, сек'].value_counts(), top_ls.iloc[i,:])

27.0    12377
24.0    10948
30.0     2396
5.0         1
36.0        1
Name: шаг балок отпускная печь, сек, dtype: int64 27.0    12342
24.0    10752
30.0     2000
31.0      333
25.0      176
32.0       42
28.0       21
30.5       21
26.0       20
33.0       14
36.0        1
5.0         1
Name: шаг балок закалочная печь, сек, dtype: int64 марка стали          30Г2
диаметр             73.02
толщина стенки       5.51
масса\nтн         11970.1
Name: 0, dtype: object
36.0    5303
30.0    1899
40.0       1
Name: шаг балок отпускная печь, сек, dtype: int64 36.0    5303
30.0    1845
31.0      50
24.0       4
9.0        1
Name: шаг балок закалочная печь, сек, dtype: int64 марка стали          30Г2
диаметр             177.8
толщина стенки        9.2
масса\nтн         10077.9
Name: 1, dtype: object
24.0    6748
30.0    1059
27.0     302
25.0      26
Name: шаг балок отпускная печь, сек, dtype: int64 24.0    6748
30.0     958
27.0     302
31.0      97
25.0      25
32.0       5
Name: шаг балок закало

In [33]:
answ = find_mark_typesize('13ХФА', 114, 6)
answ.shape

(602, 23)

In [34]:
# answ['марка стали'].value_counts()

In [35]:
answ['толщина стенки'].value_counts()

6.0    602
Name: толщина стенки, dtype: int64

In [36]:
answ.sort_values('Дата термообработки', ascending=False).head(1).to_excel('114x6 13hfa.xlsx')
answ.sort_values('Дата термообработки', ascending=False).head(1)

,2 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",Дата термообработки,3 зона по ВТР закалка,диаметр,№ плавки,масса тн,"НД (гост, ту)",1 зона по ВТР закалка,3 зона ВТР и уставка отпуск,№ партии,1 зона ВТР и уставка отпуск,t˚ C трубы после ванны,марка стали,t˚ C трубы после спреера,толщина стенки,4 зона ВТР и уставка отпуск,"Скорость прохождения трубы через спрейер, м/с",5 зона ВТР и уставка отпуск,"время выдержки в закалочной ванне, сек.",№ пакета,2 зона по ВТР закалка,"шаг балок закалочная печь, сек"
12638,715,27,2017-09-03,870,114.0,4460,3.88,ТУ 15,860,701,709046,715,NaN,13ХФА,47.5,6.0,701,0.47,686,,101674596,860,29


In [241]:
plan_may = pd.read_excel('D:\\Repo\\PycharmProjects\\gitPNTZ\\tmp_files\\план на май\\план_на_май_ПНТЗ.xlsx')
plan_may = plan_may[['Сортамент|Марка стали','Сортамент|Диам1', 'Сортамент|Стен1']]
plan_may.columns = [['марка стали', 'диаметр', 'толщина стенки']]
plan_may.drop_duplicates(inplace=True)
plan_may.reset_index(drop=True, inplace=True)

In [244]:
def close_value(df, row, value):
    s_close = df[row].value_counts().index[0]
    min_delta = np.abs(s_close-value)
    for s_ in df[row].value_counts().index:
        delta = np.abs(s_-value)
        if delta<min_delta:
            min_delta = delta
            s_close = s_
    return s_close

In [159]:
modes_old.to_excel('modes_old.xlsx')

In [251]:
answ = find_close_sort('37Г2Ф', 60, 5)
answ.sort_values('Дата термообработки', ascending=False).head(1)

(1, 26)
37Г2Ф 60 5


,"шаг балок закалочная печь, сек",время начала т/о,"шаг балок отпускная печь, сек","НД (гост, ту)","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после ванны,№ пакета,3 зона по ВТР закалка,D/S,2 зона по ВТР закалка,3 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,Дата термообработки,4 зона ВТР и уставка отпуск,1 зона по ВТР закалка,марка стали,"кол-во, шт.",1 зона ВТР и уставка отпуск,диаметр,"время выдержки в закалочной ванне, сек.",плавка,2 зона ВТР и уставка отпуск,толщина стенки,t˚ C трубы после спреера,"шаг балок закалочная/отпускная печи, сек",№ партии
132658,24,2-55,24,633-80,"0,7",НЕТ,403379663,860,"60,3Х5",850,673,656,2017-10-18,673,850,37Г2Ф,34,685,60.3,НЕТ,7F6803,685,5.0,65-75,24/24,710294


In [ ]:
def 

In [263]:
def find_close_sort(mark, d, s):
    answ = df.copy()
    if answ[answ['марка стали']==d].shape[0]==0:
        mark = mark.split('-')[0]
    answ = answ[answ['марка стали']==mark]
    if answ[answ['диаметр']==d].shape[0]!=0:
        answ = answ[answ['диаметр']==d]
        s_close = close_value(answ, 'толщина стенки',s)
        answ = answ[answ['толщина стенки']==s_close]
    elif answ[answ['толщина стенки']==s].shape[0]!=0:
        answ = answ[answ['толщина стенки']==s]
        d_close = close_value(answ, 'диаметр',d)
#         print(answ.shape)
        answ = answ[answ['диаметр']==d_close]
    return answ 

In [264]:
modes_old = pd.DataFrame()

for i in plan_may.index:
    mark = plan_may.iloc[i, 0]
    d = plan_may.iloc[i,1]
    s = plan_may.iloc[i,2]
    s = s.round(2)
#     print(mark, d, s)
    tmp = find_mark_typesize(mark, d, s)
    if tmp.shape[0]!=0:
        modes_old = pd.concat([modes_old, tmp.sort_values('Дата термообработки', ascending=False).head(1)])
    else:
        tmp = pd.DataFrame({
            '0_марка стали': [mark],
            '0_диаметр':[d],
            '0_толщина стенки':[s]})
        tmp_right = find_close_sort(mark, d, s)
        st = tmp_right.sort_values('Дата термообработки', ascending=False).head(1)
        st.reset_index(drop=True, inplace=True)
        tmp = pd.concat([tmp,st], axis=1)
        modes_old = pd.concat([modes_old, tmp])
#     print(mark, d, s)

(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)
(1, 3) (1, 26)
(1, 29)


In [266]:
modes_old.to_excel('modes_old.xlsx')

In [135]:
find_close_sort('13ХФА', 114, 6)

NameError: name 'find_close_sort' is not defined